In [39]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

import qiskit
from qiskit import transpile
from qiskit.test.mock import *
from qiskit.tools.jupyter import *   #call the name of the fake backend to get an overview of all properties as inline window
from qiskit.tools.visualization import  plot_gate_map
from qiskit.visualization import plot_coupling_map
from qiskit.transpiler import CouplingMap

import networkx as nx
import itertools
import time
import os
import sys
import csv
nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from maxcut import *
from qaoa import *
from TopologyFunctions import *

In [92]:
########################################################################################
# Fixed Parameters
########################################################################################

# initial parameters for QAOA: needed to define th circuit. The actual values are only important if the algorithm is executed
initial=[0.1, 0.1]

# optimization level for qiskit transpilation
opt_level=3

#number of compilation runs to average over
comp_averages = 20

In [93]:
########################################################################################
# Define Backends: IBM-Q Topology
########################################################################################

# Gate-set: Take from IBM-Q Brooklyn Backend. 
backend = FakeBrooklyn()
config = backend.configuration()
gate_set=config.basis_gates

# Create Copuling map
# scheme for building up larger coupling maps: start with one unit cell, then add 1 column, 1 row, 1 column again, etc. The shape is kept close to quadratic. --> later
# start with Topology similar to IBM-Q Washington
n_rows = 6
n_cols = 3
cmap = create_heavy_hex_IBMQ(n_rows, n_cols)
#n_qubits = len(set(sum(cmap, [])))

#cmap_plot=get_coupling_map_single_heavy_hex(cmap)
#qubit_coordinates=get_qubit_coordinates_heavy_hex(n_rows, n_cols, 1)
#plot_coupling_map(n_qubits, qubit_coordinates, cmap_plot, figsize=(10,10))

### IBM-Q Topology with fixed standard connectivity

In [81]:
########################################################################################
# Variable Parameters
########################################################################################

# problem size = number of nodes in the graph = number of qubits
problem_sizes = np.linspace(10, 100, 10).astype(int)

# graph density = number of edges / maximum number of edges (=all-to-all)
graph_densities = np.linspace(0.5, 0.9, 5)

# number of qaoa layers
qaoa_layers = np.array([1])#np.linspace(1, 10, 10)


In [ ]:
depths = np.zeros((len(problem_sizes), len(graph_densities), len(qaoa_layers)))
depth_stds = np.zeros((len(problem_sizes), len(graph_densities), len(qaoa_layers)))
times = np.zeros((len(problem_sizes), len(graph_densities), len(qaoa_layers)))
time_stds = np.zeros((len(problem_sizes), len(graph_densities), len(qaoa_layers)))

for i in range(len(qaoa_layers)):
    p = qaoa_layers[i]
    print('qaoa layers: ', p)
    for d in range(len(graph_densities)):
        density = graph_densities[d]
        print('graph density: ', density)
        for n in range(len(problem_sizes)):
            num_nodes = problem_sizes[n]
            print('problem_size: ', num_nodes)

            G = generate_graph_from_density(num_nodes, density)
            H = get_qubo_maxcut(G)

            theta=np.repeat(initial, p)
            qc_qaoa = get_simple_circ_qaoa_from_qubo(H, theta)

            depths_qiskit_tmp = []
            times_tmp = []
            for _ in range(comp_averages):
                start = time.time()
                transpiled_circ=transpile(qc_qaoa, basis_gates=gate_set,coupling_map=cmap, optimization_level=opt_level)
                end = time.time()
                times_tmp.append(end-start)
                depths_qiskit_tmp.append(transpiled_circ.depth())
            
            times[n, d, i] = np.mean(times_tmp)
            depths[n, d, i] = np.mean(depths_qiskit_tmp)
            depth_stds[n, d, i] = np.std(depths_qiskit_tmp)
            time_stds[n, d, i] = np.std(times_tmp)



filename = 'base_topology//prob_sizes_'+str(problem_sizes[0])+'_'+str(problem_sizes[-1])+'_prob_densities_'+str(graph_densities[0])+'_'+str(graph_densities[-1])+'_qaoa_layers_'+str(qaoa_layers[0])+'_'+str(qaoa_layers[-1])


with open('Custom_topologies_maxcut//'+filename+'.csv', "w", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['qaoa_layers', 'graph_density', 'num_qubits', 'circuit_depth_mean', 'circuit_depth_std', 'compilation_time_s', 'compilation_time_std_s'])
    for i in range(len(qaoa_layers)):
        for d in range(len(graph_densities)):
            for n in range(len(problem_sizes)):
                writer.writerow([qaoa_layers[i], graph_densities[d], problem_sizes[n], depths[n][d][i], depth_stds[n][d][i], times[n][d][i], time_stds[n][d][i]])
outfile.close()



### Increase connectivity
Use IBM-Q Washington (6 rows, 3 columns) as starting point

In [94]:
########################################################################################
# Variable Parameters
########################################################################################

# problem size = number of nodes in the graph = number of qubits
problem_sizes = np.array([10, 50]) #np.linspace(10, 100, 10).astype(int)

# graph density = number of edges / maximum number of edges (=all-to-all)
graph_densities = np.array([0.7]) #np.linspace(0.5, 0.9, 5)

# number of qaoa layers
qaoa_layers = np.array([1])#np.linspace(1, 10, 10)

# coupling densities
cmap_densities = np.array([0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])#np.linspace(0.1, 1.0, 10)

In [ ]:
depths_ext = np.zeros((len(problem_sizes), len(graph_densities), len(qaoa_layers), len(cmap_densities)))
depth_ext_stds = np.zeros((len(problem_sizes), len(graph_densities), len(qaoa_layers), len(cmap_densities)))
times_ext = np.zeros((len(problem_sizes), len(graph_densities), len(qaoa_layers), len(cmap_densities)))
time_ext_stds = np.zeros((len(problem_sizes), len(graph_densities), len(qaoa_layers), len(cmap_densities)))

for i in range(len(qaoa_layers)):
    p = qaoa_layers[i]
    print('qaoa layers: ', p)
    for d in range(len(graph_densities)):
        density = graph_densities[d]
        print('graph density: ', density)
        for n in range(len(problem_sizes)):
            num_nodes = problem_sizes[n]
            print('problem_size: ', num_nodes)

            G = generate_graph_from_density(num_nodes, density)
            H = get_qubo_maxcut(G)

            theta=np.repeat(initial, p)
            qc_qaoa = get_simple_circ_qaoa_from_qubo(H, theta)

            for cd in range(len(cmap_densities)):
                cmap_density = cmap_densities[cd]
                print('coupling density: ', cmap_density)

                cmap_ext = increase_coupling_density(cmap, cmap_density)

                depths_qiskit_tmp = []
                times_tmp = []
                for _ in range(comp_averages):
                    start = time.time()
                    transpiled_circ=transpile(qc_qaoa, basis_gates=gate_set,coupling_map=cmap_ext, optimization_level=opt_level)
                    end = time.time()
                    times_tmp.append(end-start)
                    depths_qiskit_tmp.append(transpiled_circ.depth())
                
                times_ext[n, d, i, cd] = np.mean(times_tmp)
                time_ext_stds[n, d, i, cd] = np.std(times_tmp)
                depths_ext[n, d, i, cd] = np.mean(depths_qiskit_tmp)
                depth_ext_stds[n, d, i, cd] = np.std(depths_qiskit_tmp)



filename = 'ext_topology//prob_sizes_'+str(problem_sizes[0])+'_'+str(problem_sizes[-1])+'_prob_densities_'+str(graph_densities[0])+'_'+str(graph_densities[-1])+'_qaoa_layers_'+str(qaoa_layers[0])+'_'+str(qaoa_layers[-1])+'_cmap_densitites_'+str(cmap_densities[0])+'_'+str(cmap_densities[-1])


with open('Custom_topologies_maxcut//'+filename+'.csv', "w", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['cmap_density', 'qaoa_layers', 'graph_density', 'num_qubits', 'circuit_depth_mean', 'circuit_depth_std', 'compilation_time_mean_s', 'compilation_time_std_s'])
    for cd in range(len(cmap_densities)):
        for i in range(len(qaoa_layers)):
            for d in range(len(graph_densities)):
                for n in range(len(problem_sizes)):
                    writer.writerow([cmap_densities[cd], qaoa_layers[i], graph_densities[d], problem_sizes[n], depths_ext[n][d][i][cd], depth_ext_stds[n][d][i][cd], times_ext[n][d][i][cd], time_ext_stds[n][d][i][cd]])
outfile.close()